In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoModelForSequenceClassification, AutoTokenizer
from peft import PeftModel
from transformers import BertForSequenceClassification # Import a specific model class if needed

# --- 1. LOAD TOKENIZER ---
# + Be sure to be consistent in using the Base Model's Tokeniser
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-0.6B")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# --- 2. LOADING THE POLICY MODEL---
policy_base_model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-0.6B")
policy_model = PeftModel.from_pretrained(policy_base_model, "phh/Qwen3-0.6B-TLDR-Lora")

print("✅ Policy Model Loaded")


# --- 3. LOAD THE REWARD MODEL ---
# This model gives you the score.
try:
    reward_model = AutoModelForSequenceClassification.from_pretrained("CarperAI/openai_summarize_tldr_rm_checkpoint")
except ValueError:
    reward_model = BertForSequenceClassification.from_pretrained("CarperAI/openai_summarize_tldr_rm_checkpoint")


reward_model.eval() # Set to evaluation mode
# reward_model.to("cuda")

print("✅ Reward Model Loaded")

In [ ]:
# A simple prompt to test with
prompt_text = "Human: Write a short, upbeat summary of the movie 'The Matrix'.\n\nAssistant:"

# Tokenize the prompt
prompt_inputs = tokenizer(prompt_text, return_tensors="pt") #.to("cuda")

# Generate a response
# response_tensors will include the prompt text
response_tensors = policy_model.generate(
    input_ids=prompt_inputs.input_ids,
    attention_mask=prompt_inputs.attention_mask,
    max_new_tokens=60, # Keep it short
    pad_token_id=tokenizer.pad_token_id
)

# Here we decode the generated text to see what you have
response_text = tokenizer.decode(response_tensors[0], skip_special_tokens=True)
print("\n--- Generated Response ---")
print(response_text)
with torch.no_grad(): # Use no_grad if you're only inspecting, not training yet
    outputs = policy_model(input_ids=response_tensors)
    logits = outputs.logits

print("\n--- Forward Pass Results ---")
print("✅ Success! You've got the logits.")
print(f"Shape of logits tensor: {logits.shape}")
# This shape is (batch_size, sequence_length, vocab_size)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-0.6B") # Use the tokenizer from the first cell
# reward_model = BertForSequenceClassification.from_pretrained("CarperAI/openai_summarize_tldr_rm_checkpoint")
# reward_model.eval()

prompt_text = "Human: Write a short, upbeat summary of the movie 'The Matrix'.\n\nAssistant:"
generated_summary = " A computer hacker learns from mysterious rebels about the true nature of his reality and his role in the war against its controllers."

# Combine them into the format the reward model expects
text_to_score = prompt_text + generated_summary

print("--- Text being scored ---")
print(text_to_score)



# 1. Tokenize the full text using the tokenizer loaded in the first cell
reward_inputs = tokenizer(
    text_to_score,
    return_tensors="pt",
    padding=True,
    truncation=True
)



In [ ]:
reward_inputs = tokenizer(
    response_text,
    return_tensors="pt",
    padding=True,
    truncation=True
) #.to("cuda")

# Get the score!
with torch.no_grad():
    reward_score = reward_model(**reward_inputs).logits[0]

print("\n--- The Win for Today ---")
print(f"🏆 Reward Score: {reward_score.item()}")

In [ ]:
import torch
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)
from typing import Dict, Any
from dataclasses import dataclass # Import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase # Import PreTrainedTokenizerBase
from typing import List, Union # Import List and Union

# --- 1. Configuration ---
# Model and dataset paths
BASE_MODEL_NAME = "Qwen/Qwen3-0.6B"
DATASET_PATH = "/content/drive/MyDrive/MLI/mliw6/data/asian_dad_responses_2000.parquet"
OUTPUT_MODEL_PATH = "/content/drive/MyDrive/MLI/mliw6"

# Training parameters
LEARNING_RATE = 2e-5
NUM_EPOCHS = 1
BATCH_SIZE = 4 # Adjust based on your GPU memory
MAX_LENGTH = 512 # Max sequence length for tokenization

# --- 2. Load Tokenizer and Model ---
print("Loading tokenizer and model...")

# We use a tokeniser to convert text into a format the model understands.
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME, trust_remote_code=True)

# In this case, the reward model essentially serves a classifier that outputs a single score.
# We use AutoModelForSequenceClassification with num_labels=1 to achieve this.
model = AutoModelForSequenceClassification.from_pretrained(
    BASE_MODEL_NAME,
    num_labels=1, # We want a single scalar output (the reward score)
    trust_remote_code=True,
    torch_dtype=torch.bfloat16 # Use bfloat16 for memory efficiency
)

# Decoder-only models like Qwen often don't have a pad token.
# We set it to the End-Of-Sentence token for padding purposes.
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id

# Explicitly set the model's padding token ID if it's not set
if model.config.pad_token_id is None:
    model.config.pad_token_id = tokenizer.pad_token_id


print("Tokenizer and model loaded successfully.")


# --- 3. Load and Prepare the Dataset ---
print(f"Loading dataset from '{DATASET_PATH}'...")
df = pd.read_parquet(DATASET_PATH)
dataset = Dataset.from_pandas(df)

def preprocess_function(examples: Dict[str, Any]) -> Dict[str, Any]:
    """
    This function tokenizes the chosen and rejected responses for each prompt.
    It creates two separate inputs for the reward model to score.
    """
    new_examples = {
        "input_ids_chosen": [], "attention_mask_chosen": [],
        "input_ids_rejected": [], "attention_mask_rejected": [],
    }
    for prompt, chosen, rejected in zip(examples["prompt"], examples["chosen"], examples["rejected"]):
        # Format the text as a conversation turn
        text_chosen = f"Human: {prompt}\n\nAssistant: {chosen}"
        text_rejected = f"Human: {prompt}\n\nAssistant: {rejected}"

        # Tokenize both versions
        tokenized_chosen = tokenizer(text_chosen, max_length=MAX_LENGTH, truncation=True)
        tokenized_rejected = tokenizer(text_rejected, max_length=MAX_LENGTH, truncation=True)

        new_examples["input_ids_chosen"].append(tokenized_chosen["input_ids"])
        new_examples["attention_mask_chosen"].append(tokenized_chosen["attention_mask"])
        new_examples["input_ids_rejected"].append(tokenized_rejected["input_ids"])
        new_examples["attention_mask_rejected"].append(tokenized_rejected["attention_mask"])

    return new_examples

# Apply the preprocessing to the entire dataset
print("Preprocessing dataset...")
tokenized_dataset = dataset.map(preprocess_function, batched=True, num_proc=4, remove_columns=dataset.column_names)
print("Dataset preprocessing complete.")


# --- 4. Custom Reward Trainer ---
# We create a custom Trainer to implement the ranking loss function.
class RewardTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None): # <--- ADDED num_items_in_batch=None
        """
        Computes the loss for reward model training. The goal is to maximize the
        margin between the scores of the chosen and rejected responses.
        """
        # Get the scores for the chosen responses
        rewards_chosen = model(
            input_ids=inputs["input_ids_chosen"],
            attention_mask=inputs["attention_mask_chosen"]
        )[0]

        # Get the scores for the rejected responses
        rewards_rejected = model(
            input_ids=inputs["input_ids_rejected"],
            attention_mask=inputs["attention_mask_rejected"]
        )[0]

        # The loss is calculated to ensure score(chosen) > score(rejected).
        # We use the log-sigmoid function for numerical stability.
        loss = -torch.nn.functional.logsigmoid(rewards_chosen - rewards_rejected).mean()

        if return_outputs:
            return loss, {"rewards_chosen": rewards_chosen, "rewards_rejected": rewards_rejected}
        return loss

# --- Custom Data Collator ---
@dataclass
class RewardModelDataCollator:
    """
    Data collator for reward model training.
    Pads the chosen and rejected responses separately.
    """
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str] = True
    max_length: int = None
    pad_to_multiple_of: int = None

    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, Any]:
        # Separate chosen and rejected features
        chosen_features = []
        rejected_features = []
        for feature in features:
            chosen_features.append({
                "input_ids": feature["input_ids_chosen"],
                "attention_mask": feature["attention_mask_chosen"],
            })
            rejected_features.append({
                "input_ids": feature["input_ids_rejected"],
                "attention_mask": feature["attention_mask_rejected"],
            })

        # Pad chosen and rejected features separately
        padded_chosen_features = self.tokenizer.pad(
            chosen_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        padded_rejected_features = self.tokenizer.pad(
            rejected_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        # Combine back into a single dictionary with original keys
        batch = {
            "input_ids_chosen": padded_chosen_features["input_ids"],
            "attention_mask_chosen": padded_chosen_features["attention_mask"],
            "input_ids_rejected": padded_rejected_features["input_ids"],
            "attention_mask_rejected": padded_rejected_features["attention_mask"], # Corrected key
        }
        return batch


# --- 5. Training ---
print("Setting up training arguments...")

# Define the arguments for the training process
training_args = TrainingArguments(
    output_dir=OUTPUT_MODEL_PATH,
    num_train_epochs=NUM_EPOCHS,
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    logging_steps=10,
    save_strategy="epoch",
    report_to="none", # Disable wandb/tensorboard reporting for simplicity
    bf16=True, # Enable bfloat16 mixed-precision training
    remove_unused_columns=False, # Keep columns not used by the model's forward pass
)

# Instantiate our custom trainer, providing the custom data collator
trainer = RewardTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer, # Keep the tokenizer here for the Trainer's internal use if needed,
    data_collator=RewardModelDataCollator(tokenizer=tokenizer, max_length=MAX_LENGTH) # <--- Use the custom collator
)

print("--- Starting Reward Model Training ---")
trainer.train()
print("--- Training Complete ---")


# --- 6. Save the Final Model ---
print(f"Saving the trained reward model to '{OUTPUT_MODEL_PATH}'...")
trainer.save_model(OUTPUT_MODEL_PATH)
tokenizer.save_pretrained(OUTPUT_MODEL_PATH)
print("Model saved successfully. You can now use this model for PPO.")

In [ ]:
!pip install trl


In [ ]:
!pip install transformers AdamW


In [ ]:
import torch
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForCausalLM
from peft import PeftModel
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from tqdm import tqdm

# --- 1. Configuration ---
# Model and dataset paths
POLICY_BASE_MODEL = "Qwen/Qwen3-0.6B"
POLICY_ADAPTER_MODEL = "phh/Qwen3-0.6B-TLDR-Lora"
# This is the path to the directory containing your trained reward model's config and weights
REWARD_MODEL_PATH = "/content/drive/MyDrive/MLI/mliw6/checkpoint-500"
DATASET_PATH = "/content/drive/MyDrive/MLI/mliw6/data/asian_dad_responses_2000.parquet"
FINAL_MODEL_OUTPUT_PATH = "./policy_model_asian_dad_ppo"

# PPO Training parameters
LEARNING_RATE = 1.41e-5
BATCH_SIZE = 8 # Number of prompts to process at once
MINI_BATCH_SIZE = 2 # Batch size for policy updates
EPOCHS = 1 # Number of times to iterate over the dataset
KL_PENALTY = 0.2 # KL penalty coefficient (controls how much the policy can change)
GENERATION_KWARGS = { # Parameters for the .generate() method
    "max_new_tokens": 80,
    "top_k": 0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": None, # Will be set later
}

# --- 2. Load Models and Tokenizer ---
print("--- Loading Models and Tokenizer ---")

# The tokenizer must be the same for all models
tokenizer = AutoTokenizer.from_pretrained(POLICY_BASE_MODEL, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
GENERATION_KWARGS["pad_token_id"] = tokenizer.pad_token_id

# Load the policy model (the "writer" we want to train)
# We use AutoModelForCausalLMWithValueHead from TRL, which adds a value head
# for the PPO algorithm. This head helps estimate the "value" of a state.
base_model = AutoModelForCausalLM.from_pretrained(
    POLICY_BASE_MODEL,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
)
policy_model = PeftModel.from_pretrained(base_model, POLICY_ADAPTER_MODEL)
model = AutoModelForCausalLMWithValueHead.from_pretrained(policy_model)
print("✅ Policy Model with Value Head Loaded")

# Load the reference model (a frozen copy of the initial policy)
# This is used to calculate the KL penalty.
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained(policy_model)
print("✅ Reference Model Loaded")

# Load the reward model (the "judge")
# This is the model you just trained.
reward_model = AutoModelForSequenceClassification.from_pretrained(
    REWARD_MODEL_PATH,
    num_labels=1,
    torch_dtype=torch.bfloat16,
)
# The reward model's tokenizer is not needed as we use the main one.
print("✅ Reward Model Loaded")

# Move models to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
ref_model.to(device)
reward_model.to(device)
print(f"Models moved to device: {device}")


# --- 3. Prepare Dataset and PPO Trainer ---
print("\n--- Preparing Dataset and PPO Trainer ---")

# We only need the 'prompt' column for PPO training.
df = pd.read_parquet(DATASET_PATH)
dataset = Dataset.from_pandas(df[["prompt"]])

def tokenize_prompt(example):
    """Tokenizes the prompt text."""
    # We format the prompt to guide the model's generation.
    formatted_prompt = f"Human: {example['prompt']}\n\nAssistant:"
    tokenized = tokenizer(formatted_prompt, truncation=True, max_length=512)
    tokenized["query"] = tokenizer.decode(tokenized["input_ids"])
    return tokenized

dataset = dataset.map(tokenize_prompt)
dataset.set_format(type="torch")

# Instantiate the PPOTrainer
# We pass the configuration arguments directly to the trainer and handle the
# tokenizer separately to avoid versioning errors.
ppo_trainer = PPOTrainer(
    model=model,
    ref_model=ref_model,
    dataset=dataset,
    learning_rate=LEARNING_RATE,
    batch_size=BATCH_SIZE,
    mini_batch_size=MINI_BATCH_SIZE,
    kl_penalty=KL_PENALTY,
)
# Manually set the tokenizer on the trainer object. This is a robust way
# to handle different TRL versions, as some don't accept it in the constructor.
ppo_trainer.tokenizer = tokenizer

print("✅ PPO Trainer Initialized")


# --- 4. The PPO Training Loop ---
print("\n--- Starting PPO Training ---")
progress_bar = tqdm(range(ppo_trainer.dataloader.num_batches * EPOCHS), desc="PPO Training")

for epoch in range(EPOCHS):
    for batch in ppo_trainer.dataloader:
        query_tensors = batch["input_ids"]

        # 1. ROLLOUT: Generate responses from the policy model
        response_tensors = ppo_trainer.generate(
            query_tensors,
            return_prompt=False, # We only want the generated part
            **GENERATION_KWARGS,
        )
        batch["response"] = ppo_trainer.tokenizer.batch_decode(response_tensors)

        # 2. EVALUATION: Get reward scores from the judge
        # We combine the prompt (query) and the generated response to score them.
        texts_to_score = [q + r for q, r in zip(batch["query"], batch["response"])]
        reward_inputs = ppo_trainer.tokenizer(texts_to_score, padding=True, truncation=True, return_tensors="pt").to(device)

        with torch.no_grad():
            # The reward is a single scalar score for each response.
            reward_scores = reward_model(**reward_inputs).logits

        rewards = [score for score in reward_scores]

        # 3. OPTIMIZATION: Perform a PPO step
        # This is where TRL's magic happens. The trainer handles the complex
        # calculations for advantages, ratios, and the clipped surrogate objective.
        stats = ppo_trainer.step(query_tensors, response_tensors, rewards)

        # Log the stats
        ppo_trainer.log_stats(stats, batch, rewards)
        progress_bar.update(1)

progress_bar.close()
print("--- PPO Training Complete ---")


# --- 5. Save the Final Model ---
print(f"\nSaving the final PPO-tuned model to '{FINAL_MODEL_OUTPUT_PATH}'...")
ppo_trainer.save_model(FINAL_MODEL_OUTPUT_PATH)
# Also save the tokenizer for easy loading later
ppo_trainer.tokenizer.save_pretrained(FINAL_MODEL_OUTPUT_PATH)
print("✅ Final model saved successfully!")


In [ ]:
!pip show trl


In [ ]:
pip install trl==0.19.1

In [ ]:
import torch
import pandas as pd
from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForCausalLM
from peft import PeftModel
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from tqdm import tqdm

# --- 1. Configuration ---
# Model and dataset paths
POLICY_BASE_MODEL = "Qwen/Qwen3-0.6B"
POLICY_ADAPTER_MODEL = "phh/Qwen3-0.6B-TLDR-Lora"
# This is the path to the directory containing your trained reward model's config and weights
REWARD_MODEL_PATH = "/content/drive/MyDrive/MLI/mliw6/checkpoint-500"
DATASET_PATH = "/content/drive/MyDrive/MLI/mliw6/data/asian_dad_responses_2000.parquet"
FINAL_MODEL_OUTPUT_PATH = "./policy_model_asian_dad_ppo"

# PPO Training parameters
LEARNING_RATE = 1.41e-5
BATCH_SIZE = 8 # Number of prompts to process at once
MINI_BATCH_SIZE = 2 # Batch size for policy updates
EPOCHS = 1 # Number of times to iterate over the dataset
KL_PENALTY = 0.2 # KL penalty coefficient (controls how much the policy can change)
GENERATION_KWARGS = { # Parameters for the .generate() method
    "max_new_tokens": 80,
    "top_k": 0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": None, # Will be set later
}

# --- 2. Load Models and Tokenizer ---
print("--- Loading Models and Tokenizer ---")

# The tokenizer must be the same for all models
tokenizer = AutoTokenizer.from_pretrained(POLICY_BASE_MODEL, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
GENERATION_KWARGS["pad_token_id"] = tokenizer.pad_token_id

# Load the policy model (the "writer" we want to train)
# We use AutoModelForCausalLMWithValueHead from TRL, which adds a value head
# for the PPO algorithm. This head helps estimate the "value" of a state.
base_model = AutoModelForCausalLM.from_pretrained(
    POLICY_BASE_MODEL,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
)
policy_model = PeftModel.from_pretrained(base_model, POLICY_ADAPTER_MODEL)
model = AutoModelForCausalLMWithValueHead.from_pretrained(policy_model)
print("✅ Policy Model with Value Head Loaded")

# Load the reference model (a frozen copy of the initial policy)
# This is used to calculate the KL penalty.
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained(policy_model)
print("✅ Reference Model Loaded")

# Load the reward model (the "judge")
# This is the model you just trained.
reward_model = AutoModelForSequenceClassification.from_pretrained(
    REWARD_MODEL_PATH,
    num_labels=1,
    torch_dtype=torch.bfloat16,
)
# The reward model's tokenizer is not needed as we use the main one.
print("✅ Reward Model Loaded")

# Move models to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
ref_model.to(device)
reward_model.to(device)
print(f"Models moved to device: {device}")


# --- 3. Prepare Dataset and PPO Trainer ---
print("\n--- Preparing Dataset and PPO Trainer ---")

# We only need the 'prompt' column for PPO training.
df = pd.read_parquet(DATASET_PATH)
dataset = Dataset.from_pandas(df[["prompt"]])

def tokenize_prompt(example):
    """Tokenizes the prompt text."""
    # We format the prompt to guide the model's generation.
    formatted_prompt = f"Human: {example['prompt']}\n\nAssistant:"
    tokenized = tokenizer(formatted_prompt, truncation=True, max_length=512)
    tokenized["query"] = tokenizer.decode(tokenized["input_ids"])
    return tokenized

dataset = dataset.map(tokenize_prompt)
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "query"])

def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

# PPOConfig is the modern way to configure the trainer in recent TRL versions.
# This object bundles all training parameters cleanly.
config = PPOConfig(
    learning_rate=LEARNING_RATE,
    batch_size=BATCH_SIZE,
    mini_batch_size=MINI_BATCH_SIZE,
    kl_penalty="kl",
    adap_kl_ctrl=False, # Set to False for a fixed KL penalty
    init_kl_coef=KL_PENALTY,
    log_with="none", # Disable wandb/tensorboard reporting
)

# Instantiate the PPOTrainer using the config object.
# This is the standard approach for TRL v0.7.0 and later.
ppo_trainer = PPOTrainer(
    config=config,
    model=model,
    ref_model=ref_model,
    tokenizer=tokenizer,
    dataset=dataset,
    data_collator=collator,
)
print("✅ PPO Trainer Initialized")


# --- 4. The PPO Training Loop ---
print("\n--- Starting PPO Training ---")
# The modern PPOTrainer creates its own dataloader from the provided dataset.
progress_bar = tqdm(range(len(ppo_trainer.dataloader) * EPOCHS), desc="PPO Training")

for epoch in range(EPOCHS):
    for batch in ppo_trainer.dataloader:
        query_tensors = batch["input_ids"]

        # 1. ROLLOUT: Generate responses from the policy model
        response_tensors = ppo_trainer.generate(
            query_tensors,
            return_prompt=False, # We only want the generated part
            **GENERATION_KWARGS,
        )
        batch["response"] = ppo_trainer.tokenizer.batch_decode(response_tensors)

        # 2. EVALUATION: Get reward scores from the judge
        # We combine the prompt (query) and the generated response to score them.
        texts_to_score = [q + r for q, r in zip(batch["query"], batch["response"])]
        reward_inputs = ppo_trainer.tokenizer(texts_to_score, padding=True, truncation=True, return_tensors="pt").to(device)

        with torch.no_grad():
            # The reward is a single scalar score for each response.
            reward_scores = reward_model(**reward_inputs).logits

        rewards = [score for score in reward_scores]

        # 3. OPTIMIZATION: Perform a PPO step
        # This is where TRL's magic happens. The trainer handles the complex
        # calculations for advantages, ratios, and the clipped surrogate objective.
        stats = ppo_trainer.step(query_tensors, response_tensors, rewards)

        # Log the stats
        ppo_trainer.log_stats(stats, batch, rewards)
        progress_bar.update(1)

progress_bar.close()
print("--- PPO Training Complete ---")


# --- 5. Save the Final Model ---
print(f"\nSaving the final PPO-tuned model to '{FINAL_MODEL_OUTPUT_PATH}'...")
ppo_trainer.save_model(FINAL_MODEL_OUTPUT_PATH)
# Also save the tokenizer for easy loading later
ppo_trainer.tokenizer.save_pretrained(FINAL_MODEL_OUTPUT_PATH)
print("✅ Final model saved successfully!")
